# 구글 플레이 스토어 크롤링 (한글)

### 필수 package

In [47]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
import csv 
import requests
import pandas as pd
import re
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

### 준비 작업

In [48]:
driver = webdriver.Chrome(ChromeDriverManager().install())                      

url = 'https://play.google.com/store/apps/details?id=com.samsung.android.oneconnect&hl=ko&showAllReviews=true'

driver.get(url)

time.sleep(5)

[WDM] - Current google-chrome version is 84.0.4147
[WDM] - Get LATEST driver version for 84.0.4147
[WDM] - Driver [/Users/kyung/.wdm/drivers/chromedriver/mac64/84.0.4147.30/chromedriver] found in cache


### 스크롤링

In [49]:
# 스크롤 관련 코드
last_height = driver.execute_script("return document.body.scrollHeight")  #스크롤의 위치를 찾습니다.                

while True:
    
    for i in range(3):  #스크롤을 내리면 3번까지는 자동적으로 댓글 갱신이 되는데, 4번째에는 "더보기" 버튼이 활성화됩니다.                                                                    
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") #스크롤을 가능한 아래까지 내립니다.
        time.sleep(2)  #0.8초간 반응을 봅니다.
    
    try:
        driver.execute_script('document.getElementsByClassName("RveJvd snByac")[0].click()')  #"더보기" 버튼이 활성화되면 누릅니다.
        time.sleep(2)
    except:  #같은 조건에서 "더보기" 버튼이 활성화 되지 않으면 이 과정을 무시합니다.
        pass
        
    new_height = driver.execute_script("return document.body.scrollHeight")  #스크롤을 가능한 아래까지 내렸을 때, 댓글이 갱신되어 스크롤의 위치와 크기가 바뀌면, 다시 그 스크롤을 최대한 아래까지 내립니다.
    if new_height == last_height:   #스크롤을 가능한 아래까지 내렸을 때, 댓글이 갱신되지 않아 스크롤의 위치와 크기가 바뀌지 않는다면, 이 루프를 끝냅니다.
        break
    last_height = new_height

### 크롤링 시작

In [50]:
#크롤링 관련 코드
soup = bs(driver.page_source, 'lxml')

date_list = []
name_list = []
star_list = []
review_list = []
like_list = []

In [51]:
#날짜 크롤링
dates = soup.find_all('span', class_='p2TkOb')

for date in dates:
    date_list.append(date.text)

In [52]:
#이름 크롤링
names = soup.find_all('span', class_='X43Kjb')

for name in names:
    name_list.append(name.text)

In [53]:
#별점 크롤링
stars = soup.find_all('div', class_='pf5lIe')

for star in stars:
    z = re.findall(r'\d{1}개{1}', str(star))
    z = int(z[1][0])  #별점 개수를 숫자만 뽑아서 int로 저장
    star_list.append(z)

star_list = star_list[1:-5]  #LG ThinQ 앱 평균 별점 및 추천 앱 별점 등 댓글이 준 별점 이외의 별점은 삭제 처리

In [54]:
#댓글 크롤링
fulltext_list = []

reviews = soup.find_all('span', jsname ='bN97Pc')
fulltexts = soup.find_all('span', jsname = 'fbQN7e')

for review in reviews:
    review_list.append(review.text)
    
for fulltext in fulltexts:
    fulltext_list.append(fulltext.text)
    
for i in range(len(review_list)):
    if "전체 리뷰" in review_list[i]:
        review_list[i] = fulltext_list[i]

In [55]:
#좋아요 크롤링
likes = soup.find_all('div', class_='jUL89d y92BAb')

for like in likes:
    y = re.findall(r'\d{1}개', str(like))
    y = int(y[0][0]) #좋아요 개수를 숫자만 뽑아서 int로 저장
    like_list.append(y)

IndexError: list index out of range

In [56]:
#데이터 프레임 형성
#"LG Electronics, Inc."의 댓글이 포함된 날짜, 이름 리스트에서 "LG Electronics, inc."의 index 삭제 과정
# date_list = pd.DataFrame(date_list)
# name_list = pd.DataFrame(name_list)
# df = pd.concat([date_list, name_list], axis = 1)
# df.columns = ['Date', 'Name']

# useless = df[df['Name'] == 'Samsung Electronics Co.,  Ltd.'].index
# df = df.drop(useless)
# df = df.reset_index(drop=True)

In [57]:
#데이터 프레임 형성
star_list = pd.DataFrame(star_list)
review_list = pd.DataFrame(review_list)
# like_list = pd.DataFrame(like_list)

DF = pd.concat([star_list, review_list], axis = 1)
DF.columns = ['Score', 'Review']

In [58]:
DF.shape

(80, 2)

In [60]:
# #결과 산출
# result = pd.concat([df, DF], axis = 1)

# print(result)

In [61]:
DF.to_csv("SmartThings Review Kor.csv", encoding='utf-8', index=False)

In [62]:
driver.quit()